<h2></h2>
<h1 align="center">Universidad de Buenos Aires </h1>
<h4 align="center"> Trabajo práctico 1
<br> Natural Language Processing</h4>
<p align="right"> Por: Sevann Radhak Triztan </p>

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [3]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [4]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [5]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [6]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [7]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [8]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [9]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [59]:
idx2word[25775]

'car'

In [11]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [12]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [13]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [14]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [15]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [16]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  4703, 10870,  4333])

In [17]:
# id_compare = 4253
# print(newsgroups_train.data[id_compare])

In [18]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [19]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [20]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [21]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [22]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [23]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.


# SOLUCION

**1.** Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos. Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.

In [24]:
def print_document_details(idx, data, target_names, y_train):
    print(f'{data[idx]}')
    print(f'Category: {target_names[y_train[idx]]}')
    print()

def calculate_similarity_scores(X_train, idx):
    return cosine_similarity(X_train[idx].reshape(1, -1), X_train)[0]

def get_top_similar_indices(cossim, top_n=5):
    sorted_indices = np.argsort(cossim)[::-1][1:top_n+1]
    return [(i, cossim[i]) for i in sorted_indices]

def print_top_similar_categories(most_similar_indices, target_names, y_train):
    print('Top 5 most similar category documents: ')
    for i, similarity in most_similar_indices:
        print(f"\t- {target_names[y_train[i]]} \t- with similarity: {similarity:.4f}")

def find_similar_documents(idx, X_train, y_train, data, target_names):
    print_document_details(idx, data, target_names, y_train)
    cossim = calculate_similarity_scores(X_train, idx)
    most_similar_indices = get_top_similar_indices(cossim)
    print_top_similar_categories(most_similar_indices, target_names, y_train)

In [25]:
ids = [500, 555, 666, 814, 878]

In [26]:
for i, idx in enumerate(ids):
    print(f"Document {i+1}:")
    find_similar_documents(idx, X_train, y_train, newsgroups_train.data, newsgroups_train.target_names)
    print()

Document 1:

I got one from Microsoft tech support.

Category: comp.os.ms-windows.misc

Top 5 most similar category documents: 
	- comp.os.ms-windows.misc 	- with similarity: 0.2674
	- comp.sys.ibm.pc.hardware 	- with similarity: 0.2341
	- talk.politics.guns 	- with similarity: 0.2080
	- comp.sys.mac.hardware 	- with similarity: 0.1927
	- comp.sys.ibm.pc.hardware 	- with similarity: 0.1802

Document 2:
The subject does not describe the problem I am having very well.  Please read
on...

I am trying to write a function which creates an XtAppContext and a Widget,
displays the widget for a while, then destroys everything and returns.  The
problem is that multiple calls to this function cause a variety of problems
including (depending on which calls I make to get rid of things):

- Core Dump
- BadPixmap X Error
- Widget not unmapped


Here is a simple (C++) program I wrote to show the problem:

#include <X11/Xlib.h>
#include <Xm/Xm.h>
#include <Xm/PushB.h>

void bla()
{
        XtAppContext

### **Conclusiones:**  
En general, la vectorización y medición de similaridad funcionan bien para agrupar documentos de contenido y temática similar, especialmente en categorías técnicas o altamente específicas. Esto se observa claramente en los documentos relacionados con temas de computación y tecnología, donde los documentos más similares pertenecen a la misma categoría o a categorías estrechamente relacionadas, con valores de similaridad que varían desde moderados a altos.  

En el caso de temas más generales o transversales, como el Documento 4, que toca temas sociales o políticos, la similaridad tiende a dispersarse más, indicando que el modelo puede encontrar desafíos al tratar de agrupar estos documentos basándose únicamente en el contenido textual. Esto se refleja en valores de similaridad más bajos y en la aparición de categorías diversas entre los documentos más similares.

**2.** Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

In [33]:
def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = f1_score(y_test, y_pred, average='macro')
    return score

In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer

vectorizers = [
    TfidfVectorizer(min_df=2),
    TfidfVectorizer(ngram_range=(1,2)),
    TfidfVectorizer(stop_words='english', max_features=5000, max_df=0.5),
    CountVectorizer(min_df=2),
    CountVectorizer(ngram_range=(1, 3), max_features=10000)
]

models = [
    LogisticRegression(C=1.0, penalty='l2', solver='liblinear'),
    SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, max_iter=1000),
    RandomForestClassifier(n_estimators=100, max_depth=None),
    ComplementNB(alpha=0.5),
    ComplementNB(alpha=1, norm=True, fit_prior=False, class_prior=None)
]

In [54]:
best_score = 0
best_vectorizer = None
best_model = None

for vectorizer in vectorizers:
    X_train_vect = vectorizer.fit_transform(newsgroups_train.data)
    X_test_vect = vectorizer.transform(newsgroups_test.data)

    for model in models:
        score = evaluate_model(model, X_train_vect, y_train, X_test_vect, y_test)
        print(f'Vectorizer: {vectorizer}, Model: {model}, F1 Score: {score}')
        if score > best_score:
            best_score = score
            best_vectorizer = vectorizer
            best_model = model

print(f'\nBest Vectorizer: {best_vectorizer}, Best Model: {best_model}, Best F1 Score: {best_score}')

Vectorizer: TfidfVectorizer(min_df=2), Model: LogisticRegression(solver='liblinear'), F1 Score: 0.6594410404729144
Vectorizer: TfidfVectorizer(min_df=2), Model: SGDClassifier(), F1 Score: 0.6782524993248341
Vectorizer: TfidfVectorizer(min_df=2), Model: RandomForestClassifier(), F1 Score: 0.5675894032630928
Vectorizer: TfidfVectorizer(min_df=2), Model: ComplementNB(alpha=0.5), F1 Score: 0.6980382854233241
Vectorizer: TfidfVectorizer(min_df=2), Model: ComplementNB(alpha=1, fit_prior=False, norm=True), F1 Score: 0.6896750732335452
Vectorizer: TfidfVectorizer(ngram_range=(1, 2)), Model: LogisticRegression(solver='liblinear'), F1 Score: 0.6331337743816635
Vectorizer: TfidfVectorizer(ngram_range=(1, 2)), Model: SGDClassifier(), F1 Score: 0.6812257369220489
Vectorizer: TfidfVectorizer(ngram_range=(1, 2)), Model: RandomForestClassifier(), F1 Score: 0.5705819130491745
Vectorizer: TfidfVectorizer(ngram_range=(1, 2)), Model: ComplementNB(alpha=0.5), F1 Score: 0.6922295046328111
Vectorizer: TfidfV

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

In [69]:
tfidfvect = TfidfVectorizer(min_df=2, stop_words='english')
X_train = tfidfvect.fit_transform(newsgroups_train.data)

In [80]:
def find_similar_words(word_idx, X_transposed, feature_names, top_n=5):
    word = feature_names[word_idx]
    print(f"Word: {word}")
    cossim = cosine_similarity(X_transposed[word_idx], X_transposed)[0]
    most_similar_indices = np.argsort(cossim)[::-1][1:top_n+1]
    print("Top 5 most similar words:")
    for idx in most_similar_indices:
        similar_word = feature_names[idx]
        print(f"\t- {similar_word} \t- with: {cossim[idx]:.4f}")
    print()

In [85]:
X_train_transposed = X_train.T
feature_names = tfidfvect.get_feature_names_out()

In [83]:
def get_word_index(word):
    return tfidfvect.vocabulary_.get(word)

In [105]:
words = ['hospital', 'travel', 'gun', 'london', 'payment']
word_indices = [get_word_index(word) for word in words]

for word_idx in word_indices:
    find_similar_words(word_idx, X_train_transposed, feature_names)

Word: hospital
Top 5 most similar words:
	- curse 	- with: 0.4549
	- hospitalization 	- with: 0.2377
	- copayment 	- with: 0.2063
	- copayments 	- with: 0.2017
	- medical 	- with: 0.2000

Word: travel
Top 5 most similar words:
	- transferable 	- with: 0.3328
	- kegs 	- with: 0.2386
	- airline 	- with: 0.2084
	- cj 	- with: 0.1995
	- scrabble 	- with: 0.1994

Word: gun
Top 5 most similar words:
	- guns 	- with: 0.3792
	- crime 	- with: 0.2475
	- handgun 	- with: 0.2465
	- firearms 	- with: 0.2381
	- homicides 	- with: 0.2285

Word: london
Top 5 most similar words:
	- brakdown 	- with: 0.4776
	- jumpin 	- with: 0.4776
	- collectable 	- with: 0.3191
	- rolling 	- with: 0.3161
	- sleeve 	- with: 0.3138

Word: payment
Top 5 most similar words:
	- expense 	- with: 0.4145
	- goods 	- with: 0.3998
	- receipt 	- with: 0.3886
	- defective 	- with: 0.3514
	- satisfactory 	- with: 0.3388



**Conclusiones:**  

Basado en los resultados obtenidos al estudiar las palabras seleccionadas para calcular las palabras más similares, observamos que, en la mayoría de los casos, las palabras similares capturan ciertos aspectos semánticos relacionados.  

Por ejemplo:

- **Hospital:** Las palabras similares incluyen términos como "hospitalization" y "medical", lo cual es coherente sintácticamente y refleja su uso en contextos médicos.
- **Travel:** Aquí vemos términos como "airline" y "scrabble", donde "airline" es semánticamente relevante pero "scrabble" parece ser una anomalía.
- **Gun:** Las palabras similares incluyen términos como "crime" y "firearms", que son relevantes sintácticamente en el contexto de armas y crímenes.
- **London:** Las palabras similares como "brakdown" y "jumpin" pueden no ser sintácticamente precisas, sugiriendo limitaciones en la precisión semántica del modelo para nombres propios.
- **Payment:** Aquí vemos términos como "expense" y "receipt", que son coherentes sintácticamente en el contexto de transacciones financieras.